<a href="https://colab.research.google.com/github/dbwofla11/DaconBaseLine_LLM-gemma-7b-/blob/master/TransFormer_Base1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp